<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Краткий-обзор-данных" data-toc-modified-id="Краткий-обзор-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Краткий обзор данных</a></span></li><li><span><a href="#Подготовка-признаков-для-обучения" data-toc-modified-id="Подготовка-признаков-для-обучения-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка признаков для обучения</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Нахождение-R2-на-нешифрованном-массиве" data-toc-modified-id="Нахождение-R2-на-нешифрованном-массиве-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Нахождение R2 на нешифрованном массиве</a></span></li><li><span><a href="#Создание-алгоритма" data-toc-modified-id="Создание-алгоритма-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Создание алгоритма</a></span></li><li><span><a href="#Итог" data-toc-modified-id="Итог-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Итог</a></span></li></ul></li><li><span><a href="#Итог" data-toc-modified-id="Итог-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итог</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Для начала сделаем импорт нужных библиотек:

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None
from sklearn.metrics import r2_score

Импортируем нужный датасет:

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

### Краткий обзор данных

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df.sample(random_state=42)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
1501,1,28.0,56100.0,0,0


Проверим количество дубликатов:

In [7]:
df.duplicated().sum()

153

В данных присутствуют дубликаты, целью задания не является построение модели с наилучшим значением метрики, а работа с целым числом 5000 намного приятнее, поэтому не будем торопится с удалением дубликатов. К тому же данные имеют не настолько большой диапазон, чтобы считать, что это дубликаты.

Как видно столбцы Возраст и Зарплата имеют данные float. Возможно,лучше было бы перенести их в значение int, чтобы сделать таблицу более низкой по памяти.

In [8]:
df['Возраст'].value_counts()

19.0    223
25.0    214
31.0    212
26.0    211
22.0    209
27.0    209
32.0    206
28.0    204
29.0    203
30.0    202
23.0    202
21.0    200
20.0    195
36.0    193
33.0    191
24.0    182
35.0    179
34.0    177
37.0    147
39.0    141
38.0    139
41.0    129
18.0    117
40.0    114
42.0     93
43.0     77
44.0     74
45.0     73
46.0     60
47.0     47
49.0     37
50.0     27
48.0     26
52.0     22
51.0     21
53.0     11
55.0      9
54.0      7
56.0      5
59.0      3
60.0      2
58.0      2
57.0      2
62.0      1
65.0      1
61.0      1
Name: Возраст, dtype: int64

In [9]:
df['Зарплата'].value_counts()

45800.0    29
37100.0    28
43200.0    27
41500.0    27
46800.0    26
           ..
68900.0     1
59700.0     1
59000.0     1
68000.0     1
64400.0     1
Name: Зарплата, Length: 524, dtype: int64

Как видим, все значения можно перенести в тип int

In [10]:
df['Возраст'] = np.int64(df['Возраст'])

In [11]:
df['Зарплата'] = np.int64(df['Зарплата'])

### Подготовка признаков для обучения

In [12]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [13]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=42)

In [14]:
features_train.shape

(3750, 4)

In [15]:
features_test.shape

(1250, 4)

In [16]:
target_train.shape

(3750,)

In [17]:
target_test.shape

(1250,)

## Умножение матриц

Как создать произволбную обратимую матрицу? numpy.random.normal() 

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула предсказания с минимальной функцией потерь

$$
a = X(X^T X)^{-1} X^T y
$$

Нужно доказать:

Если

$$
w_{1} = (X^T × X)^{-1} × X^T × y = ((XP)^T × XP)^{-1} × (XP)^T × y 
$$

И

$$
a_{1} = XP × ((XP)^T× XP)^{-1} × (XP)^T × y
$$


где P - является обратимой матрицей

В итоге нужно доказать:

$$
a = a_{1}
$$

**Ответ:** Не изменится

**Обоснование:** 

- Еще раз обратимся к 

$$
a_{1} = XP × ((XP)^T × XP)^{-1} × (XP)^T × y
$$

- Раскроем более четко, что нужно доказать:

$$
X(X^T X)^{-1} X^T y = XP × ((XP)^T × XP)^{-1} × (XP)^T × y
$$


Свойство матриц, нужные для док-ва:

$$
(AB)^{-1} = B^{-1}A^{-1}
$$

$$
(AB)^T = B^TA^T
$$

$$
ABC = (AB)C=A(BC)
$$

$$
EA = AE = A
$$

$$
AA^{-1} = A^{-1}A=E
$$

Воспользуемся данным свойствами матриц:

$$ 
a_{1}=XP × (XP)^{-1} × ((XP)^T)^{-1} × (XP)^T × y = XPP^{-1}X^{-1} × ((XP)^T)^{-1} × (XP)^T × y = XEX^{-1} × ((XP)^T)^{-1} × (XP)^T × y 
$$

$$
XEX^{-1} × ((P^TX^T)^{-1} × P^TX^T × y
$$

$$
XEX^{-1} × (X^T)^{-1} × (P^T)^{-1} × P^TX^T × y 
$$

$$
XEX^{-1} × (X^T)^{-1} × EX^T × y 
$$

$$
XX^{-1} × (X^T)^{-1} × X^T × y
$$

$$
a_{1} = XX^{-1} × (X^T)^{-1} × X^T × y = X(X^TX)^{-1}X^Ty
$$

$$
a = X(X^T X)^{-1} X^T y
$$

$$
a_{1} = a
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
....
$$

$$
a' = X(P^T (X^T X) P)^{-1} (XP)^T y
$$

$$
(X^T X)
$$

Последняя матрица - это одна квадратная матрица, которая может быть подвергнута первому свойству матриц

Воспользуемся первым свойством матриц, действительным только для квадратных (а матрица X в степени T умноженная на X является таковой) 

$$
a' = X P^{-1} (X^T X)^{-1} (P^T)^{-1} (XP)^T y
$$

$$
a' = X (P^{-1}) (X^T X)^{-1} (P^T)^{-1} P^TX^T y
$$

$$
a' = X (P^{-1}) (X^T X)^{-1} EX^T y
$$

Воспользуемся пятым свойством матриц

$$
a' = X (P^{-1}) (X^T X)^{-1} X^T y
$$

Можно заметить, что права часть уравнения после P в -1 степени является нужным нам w

$$
w' = P^{-1} w
$$

$$
a' = X(P^{-1})w
$$

Что и требовалось доказать

## Алгоритм преобразования

**Алгоритм**

На основе выведенного выше док-ва можно выдвинуть следующий алгоритм:
1. Создадим матрицу, соответствующую размерам исходной матрице.
2. Проверим её на необратимость
3. Умножим исходную матрицу на новую матрицу.
4. Применение алгоритма на получивщейся матрице.

**Обоснование**

Выведенное выше докозательства является подтверждением того, что умножение исходной матрице на преобразованную не изменит качество предсказания.

## Проверка алгоритма

### Нахождение R2 на нешифрованном массиве

In [18]:
model = LinearRegression()
model.fit(features_train, target_train)
print('R2 =',r2_score(target_test, model.predict(features_test)))
print('W-coef',model.coef_)

R2 = 0.4254778535754761
W-coef [ 4.92432127e-03  3.51527195e-02 -2.45798536e-07 -1.49140097e-02]


In [19]:
no_matrix_model = r2_score(target_test, model.predict(features_test)) #переменная для итоговой таблицы

### Создание алгоритма

Создадим алгоритм для преобразования матрицы признаков:

In [20]:
def encoded_matrix(features):
    n = features.shape[1]
    matrix = np.random.normal(loc = 30, scale = 10, size=(n,n)) 
    while np.linalg.det(matrix) == 0:
        matrix = np.random.normal(loc = 30, scale = 10, size=(n,n))
    encoded_matrix = features @ matrix
    encoded_matrix.columns = [features.columns]
    return encoded_matrix

In [21]:
new_features = encoded_matrix(features)

In [22]:
new_features

,Пол,Возраст,Зарплата,Члены семьи
0,1.320090e+06,1.954372e+06,1.907499e+06,1.749775e+06
1,1.011983e+06,1.497689e+06,1.461519e+06,1.341179e+06
2,5.593898e+05,8.277493e+05,8.077036e+05,7.413243e+05
3,1.109260e+06,1.642730e+06,1.603548e+06,1.470487e+06
4,6.949217e+05,1.028585e+06,1.003813e+06,9.210346e+05
...,...,...,...,...
4995,9.501050e+05,1.406656e+06,1.372932e+06,1.259353e+06
4996,1.394166e+06,2.064401e+06,2.015047e+06,1.848112e+06
4997,9.019153e+05,1.335553e+06,1.303643e+06,1.195569e+06
4998,8.701823e+05,1.288427e+06,1.257588e+06,1.153417e+06


Как видим в датафрейме получились абсолютно иные значения

Разделим данные для обучающей и тестовой выборок:

In [23]:
features_train_matrix, features_test_matrix, target_train_matrix, target_test_matrix = train_test_split(new_features, target, test_size=0.25, random_state=42)

Обучим на них модель

In [24]:
model_m=LinearRegression()
model_m.fit(features_train_matrix, target_train_matrix)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:
print('R2 =',r2_score(target_test_matrix, model_m.predict(features_test_matrix)))
print('W-coef', model_m.coef_)

R2 = 0.4254778535755155
W-coef [ 0.00081293 -0.00245895  0.00068913  0.00138273]


In [26]:
matrix_model = r2_score(target_test_matrix, model_m.predict(features_test_matrix)) #переменная для итоговой таблицы

### Итог

Создадим датафрейм, чтобы заказчику было удобнее сравнить данные:

In [27]:
final_df = pd.DataFrame(data = [no_matrix_model, matrix_model], 
                        columns=['R2'], 
                        index=['Линейная регрессия', 'Линейнай регрессия с преобразованными признаками'])

In [28]:
final_df

,R2
Линейная регрессия,0.425478
Линейнай регрессия с преобразованными признаками,0.425478


## Итог

- Умножение признаков на обратимую матрицу не несет вреда для линейной регрессии.
- Данный алгоритм может быть использован для кодирования данных